In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from codac import Interval, IntervalVector, CtcDist, Function,CtcFunction
from collections import deque
import networkx as nx
from scipy.optimize import minimize, rosen, rosen_der

c:\Users\thoma\.conda\envs\map588\lib\site-packages\numpy\core\getlimits.py:493: UserWarning: Signature b'\xcc\xcc\xcc\xbd' for <class 'numpy.float32'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)
c:\Users\thoma\.conda\envs\map588\lib\site-packages\numpy\core\_machar.py:295: RuntimeWarning: overflow encountered in divide
  xmax = xmax / (xmin*beta*beta*beta)


In [2]:
f= open("graph_02.txt", mode="r")
li = []
for line in f:
    li.append(line)

In [3]:
f.close()

In [4]:
dico_constrains = {}
dico_interval = {}
for elem in li:
    i = int(elem[6:8].split(" ")[-1])-1
    j = int(elem[16:18].split(" ")[-1])-1
    mini = float(elem[21:42])
    maxi = float(elem[45:-1])
    dico_constrains[(i,j)] = (maxi + mini)/2
    dico_interval[(i,j)] = [mini, maxi]

# Evolution

In [ ]:
def c_in(X,dico_interval):
    count = 0
    for key, value in dico_interval.items():
        i,j = key
        mini,maxi = value
        if (np.linalg.norm(X[i]-X[j]) > mini and np.linalg.norm(X[i]-X[j]) < maxi):
            count += 1
    return count

In [ ]:
def error(X,dico_interval):
    li = []
    for key, value in dico_interval.items():
        i,j = key
        mini,maxi = value
        li.append([np.linalg.norm(X[i]-X[j]), mini, maxi])
    return li

In [ ]:
def cost_indiv(X, dico_interval):
    cout = 0
    for key, value in dico_interval.items():
        i,j = key
        mini, maxi = value
        mean = (maxi+mini)/2
        dist = np.linalg.norm(X[i]-X[j])
        cout += np.max([(maxi**2 -dist**2)/maxi**2, (dist**2 - mini**2)/mini**2, 0])
        #cout += (dist**2  - mean**2)**2 
    return cout

In [ ]:
def cost(X,dico_interval):
    n = X.shape[0]
    li_cost = np.zeros(n)
    for i in range(n):
        li_cost[i] = cost_indiv(X[i],dico_interval)
    return li_cost

In [ ]:
def select(X,cost, frac = 0.7):
    ordering = np.argsort(cost)
    n = int(frac*ordering.shape[0])
    X_child = X[ordering[:n]]
    return X_child


In [ ]:
def variation(X,i):
    atoms = np.random.choice([i for i in range(63)],size=X.shape[0], replace=True)
    X[:,atoms,:] += np.random.normal(loc=0, scale=0.02,size=(X.shape[0],3))
    return X


In [ ]:
def remplacement(X,X_child,cost_X, cost_child):
    n = X.shape[0]
    cost_tot = np.concatenate([cost_X,cost_child])
    ordering = np.argsort(cost_tot)[:n]
    X_tot = np.concatenate([X, X_child])
    return X_tot[ordering], cost_tot[ordering]


In [ ]:
X = np.random.normal(loc=0, scale= 10,size=(200,63,3))
cost_X = cost(X, dico_interval)
mean = np.mean(cost_X)
N_iter = 1000
n = 0
while n<N_iter:
    if (n%10 == 0):
        print(n)
    X_child =  select(X, cost_X, frac=0.7)
    X_child = variation(X_child,i)
    cost_child = cost(X_child, dico_interval)
    X, cost_X = remplacement(X,X_child, cost_X,cost_child)
    n += 1


In [ ]:
c_in(X,dico_interval)

In [ ]:
cost_X[0]

In [ ]:
li = error(X[0], dico_interval)

In [ ]:
li

# Explore

In [ ]:
exist = True
count = 0
for i in range(3,63):
    if not (links[i-3,i] and links[i-2,i] and links[i-1,i]):
        count +=1

# Codac

In [ ]:
M = 63

In [ ]:
help(Interval)

In [ ]:
li_f = []
li_interval = []
for (l,k), [mini,maxi] in dico_interval.items():
    i = 3*l 
    j = 3*k 
    li_f.append(Function(f"X[{M*3}]", f"sqrt((X[{i}]-X[{j}])^2+(X[{i}+1]-X[{j}+1])^2+(X[{i}+2]-X[{j}+2])^2)"))
    li_interval.append((mini, maxi))



In [ ]:
li_ctf = []
X = IntervalVector(M*3,(-10,10))
for (l,k), [mini,maxi] in dico_interval.items():
    i = 3*l 
    j = 3*k
    ctc_f = CtcFunction(Function(f"X[{M*3}]", f"sqrt((X[{i}]-X[{j}])^2+(X[{i}+1]-X[{j}+1])^2+(X[{i}+2]-X[{j}+2])^2)"), Interval(mini,maxi))
    li_ctf.append(ctc_f)
for i in range(M-3):
    ctc_f = CtcFunction(Function(f"X[{M*3}]", f"""sqrt((X[{i}]-X[{i+1}])^2+(X[{i}+1]-X[{i+1}+1])^2+(X[{i}+2]-X[{i+1}+2])^2) * 
                                 sqrt((X[{i+1}]-X[{i+2}])^2+(X[{i}+1]-X[{i+2}+1])^2+(X[{i}+2]-X[{i+2}+2])^2) * 
                                 """), Interval(0,+oo))
    li_ctf.append(ctc_f)
"""for i in range(M-1):
    print(i)
    for j in range(i,M-2):
        if (i!=j):
            for k in range(j,M-3):
                if i!=k and j!=k:
                    ctc_f = CtcFunction(Function(f"X[{M*3}]", f"((X[{i}]-X[{j}])^2+(X[{i}+1]-X[{j}+1])^2+(X[{i}+2]-X[{j}+2])^2)^0.5 - ((X[{i}]-X[{k}])^2+(X[{i}+1]-X[{k}+1])^2+(X[{i}+2]-X[{k}+2])^2)^0.5 - ((X[{k}]-X[{j}])^2+(X[{k}+1]-X[{j}+1])^2+(X[{k}+2]-X[{j}+2])^2)^0.5"), 
                                        Interval(-oo,0))  
                    li_ctf.append(ctc_f)"""
    

In [ ]:
def contract(X):
    x = X
     
    for (l,k), [mini,maxi] in dico_interval.items():
        i = 3*l 
        j = 3*k 
        a = IntervalVector([x[i],x[i+1],x[i+2]])
        b = IntervalVector([x[j],x[j+1],x[j+2]])
        ctc.dist.contract(a,b,Interval(mini, maxi))
        x[i],x[i+1],x[i+2] = a
        x[j],x[j+1],x[j+2] = b
    return x

In [ ]:
def C_in(X, li_f, li_interval):
    if (empty(X)):
        return False
    in_it = True
    for i in range(len(li_f)//2):
        eval = li_f[i].eval(X)
        mini, maxi = li_interval[i]
        if ((eval.lb() < mini) or (eval.ub()>maxi)):
            #in_it = False
            return False
    return True

In [ ]:
def C_out(X,li_f, li_interval):
    out_it = False
    for i in range(len(li_f)//2):
        eval = li_f[i].eval(X)
        mini, maxi = li_interval[i]
        if ((eval.lb()> maxi) or (eval.ub() < mini)):
            out_it = True
    return out_it

In [ ]:
def get_largest(X):
    max_index = 0
    for i in range(M*3):
        if (X[max_index].diam() < X[i].diam()):
            max_index = i
    return max_index

In [ ]:
def empty(X):
    for i in range(M*3):
        if X[i].is_empty():
            return True
    return False

In [ ]:
C_in(X,li_f, li_interval)

In [ ]:
X[0]

In [ ]:
def iteration(stack, stack_acc, stack_rej, stack_unc):
    X = stack.pop()
    count +=1
    X = contract(X)
    if not empty(X):
        if (C_in(X,li_f, li_interval)):
            sol = True
            print("sol")
            stack_acc.append(X)
        else:
            if C_out(X, li_f, li_interval):
                stack_rej.append(X)
            else:
                larger_index = get_largest(X)
                if X[larger_index].diam() > tau:
                    X_left, X_right  = X.bisect(larger_index)
                    if not (empty(X_left)):
                        stack.append(X_left)
                    if not (empty(X_right)):
                        stack.append(X_right)
                else:
                    stack_unc.append(X)
    return stack, stack_acc, stack_rej, stack_unc

In [ ]:
tau = 0.5
stack = [IntervalVector(M*3,(-10,10))]
stack_acc = deque()
stack_rej = deque()
stack_unc = deque()
sol = False
count = 0
while len(stack) !=0 and (not sol):
    X = stack.pop()
    count +=1
    X = contract(X)
    if not empty(X):
        if (C_in(X,li_f, li_interval)):
            sol = True
            print("sol")
            stack_acc.append(X)
        else:
            if C_out(X, li_f, li_interval):
                stack_rej.append(X)
            else:
                larger_index = get_largest(X)
                if X[larger_index].diam() > tau:
                    X_left, X_right  = X.bisect(larger_index)
                    if not (empty(X_left)):
                        stack.append(X_left)
                    if not (empty(X_right)):
                        stack.append(X_right)
                else:
                    stack_unc.append(X)
    
    


    
        

In [ ]:
X

In [ ]:
solution = stack_acc.pop()

In [ ]:
empty(solution)

In [ ]:
a = stack.pop()
b= a.copy()

In [ ]:
a[10].diam()

In [ ]:
a

In [ ]:
b

# Lsbuild

## Clique

In [5]:
G = nx.Graph()

In [6]:
G.add_nodes_from([i for i in range(63)])

In [7]:
for (i,j), (mini, maxi) in dico_interval.items():
    G.add_edges_from([(i,j,{"mini":mini, "maxi":maxi})])

In [8]:
cliques = nx.find_cliques(G)

In [9]:
maxi_clique = max(nx.find_cliques(G), key=len)

In [10]:
sub = nx.Graph()
for node in maxi_clique:
    sub.add_node(node)

In [11]:
for (i,j), (mini, maxi) in dico_interval.items():
    if i in maxi_clique and j in maxi_clique:
        sub.add_edges_from([(i,j,{"mini":mini, "maxi":maxi})])

In [ ]:
list(sub.nodes)

In [12]:
def get_edm(nodes,t, graph):
    n = len(nodes)
    matrix = np.zeros((n,n))
    for k in range(n-1):
        for l in range(k+1,n):
            i = nodes[k]
            j = nodes[l]
            low = graph.get_edge_data(i,j)["mini"]
            up = graph.get_edge_data(i,j)["maxi"]
            matrix[k,l] = (1-t[k,l])*low + t[k,l]*up
            matrix[l,k] = (1-t[k,l])*low + t[k,l]*up
    return matrix

In [13]:
def get_x(EDM):
    U, S, Vh = np.linalg.svd(EDM)
    x = U[:,:3]
    return x

In [14]:
lamb = 1
tau = 0.1

def theta(x,i,j,c):
    return (lamb**2 *(c-np.sqrt(np.linalg.norm(x[3*i:3*(i+1)]-x[3*j:3*(j+1)])**2 + tau**2))**2 + tau**2)
def phi(x,i,j, low,up):
    value = lamb*(low - up) + theta(x,i,j,low) + theta(x,i,j,up)
    return value
def cost(x,nodes,graph):
    n = len(nodes)
    cost_value = 0
    for k in range(n-1):
        for l in range(k+1,n):
            i = nodes[k]
            j = nodes[l]
            low = graph.get_edge_data(i,j)["mini"]
            up = graph.get_edge_data(i,j)["maxi"]
            cost_value += phi(x,k,l,low,up)
    return cost_value


In [15]:
def get_n_error(X,nodes, graph):
    n = len(nodes)
    count = 0
    relative_error = []
    count_tot = 0
    for k in range(n-1):
        for l in range(k+1,n):
            i = nodes[k]
            j = nodes[l]
            count_tot += 1
            low = graph.get_edge_data(i,j)["mini"]
            up = graph.get_edge_data(i,j)["maxi"]
            dist = np.linalg.norm(X[k]-X[l])
            print(f"edge: ({i,j}")
            print(dist)
            print(f"low : {low}    up : {up}")
            print()
            if (dist>up or dist < low):
                count += 1
                relative_error.append(np.min([np.abs(dist-low)/low,np.abs(dist-up)/up]))

    return

In [16]:
t = np.ones((9,9)) * 0.7
D = get_edm(list(sub.nodes),t,sub)
x = get_x(D)
res = minimize(cost,x.flatten(),args=(list(sub.nodes),sub),method='BFGS')
X = res.x.reshape(9,3)

In [ ]:
get_n_error(X,list(sub.nodes),sub)

In [17]:
dico_index_to_X = {}
dico_X_to_index = {}
count = 0
for node in list(sub.nodes):
    dico_index_to_X[node] = count
    count += 1
count = 0
for node in list(sub.nodes):
    dico_X_to_index[count] = node
    count += 1 

In [18]:
dico_index_to_X

{18: 0, 20: 1, 21: 2, 19: 3, 22: 4, 24: 5, 25: 6, 26: 7, 23: 8}

In [19]:
def best_node(nodes_base,node_to_choose, graph):
    counter = [[] for _ in range(len(node_to_choose))]
    for (i,j) in graph.edges:
        if ((i in nodes_base) and (j not in nodes_base)):
            counter[j].append(i)
        if ((j in nodes_base) and (i not in nodes_base)):
            counter[i].append(j)
    for i in range(len(counter)):
        if (len(counter[i])>=3):
            return i, counter[i]


In [20]:
node, node_base_connected = best_node(list(sub.nodes), list(G.nodes), G)

In [21]:
X

array([[ 0.92379823, -3.5731522 , -0.09978916],
       [ 1.02014942, -1.09452151,  0.45208743],
       [ 0.5266358 , -1.48538446, -1.18061632],
       [-0.94895303, -2.13476559,  1.38241174],
       [-1.2133437 ,  0.10901311, -1.18948256],
       [ 0.4240286 ,  1.58868486,  0.84494465],
       [-1.3290413 ,  2.28799029, -1.21421349],
       [-1.01081181,  4.10579787,  0.04746002],
       [-1.35792877,  0.11808317,  0.55467378]])

In [22]:
def update_EDM(D, X, nodes_base, dico_index_to_X):
    n = len(nodes_base)
    for k in range(n-1):
        for l in range(k+1, n):
            i = dico_index_to_X[nodes_base[k]]
            j = dico_index_to_X[nodes_base[l]]
            D[i,j] = np.linalg.norm(X[i] - X[j])
            D[j,i] = np.linalg.norm(X[i] - X[j])
    return D

In [24]:
print(G.get_edge_data(10,18))
print(G.get_edge_data(10,20))
print(G.get_edge_data(10,21))

{'mini': 3.280604499173894, 'maxi': 3.346886033315147}
{'mini': 3.185788056352778, 'maxi': 3.250153913278571}
{'mini': 2.469650991537058, 'maxi': 2.519547971363112}


## Compute 10th atom

In [25]:
def get_f_ctc(node_base_connected, node_to_solve, graph):
    li_ctc = []
    li_constrains = []
    li_f = []
    for atom in node_base_connected:
        low = graph.get_edge_data(node_to_solve,atom)["mini"]
        up = graph.get_edge_data(node_to_solve,atom)["maxi"]
        li_constrains.append([low,up])
        center = X[dico_index_to_X[atom]]
        f = Function("x[3]", f"sqrt(sqr(x[0]-{center[0]})+sqr(x[1]-{center[1]})+sqr(x[2]-{center[2]}))")
        li_f.append(f)
        contractor = CtcFunction(f, Interval(low,up))
        li_ctc.append(contractor)
    return li_ctc,li_f, li_constrains


In [26]:
li_ctc, li_f, li_constrains = get_f_ctc(node_base_connected, node, G)

In [ ]:
len(li_f)

In [27]:
def contract(x,li_ctc):
    for ctc in li_ctc:
        ctc.contract(x)
    

In [28]:
def empty(X):
    for i in range(3):
        if X[i].is_empty():
            return True
    return False

In [29]:
def C_in(X, li_f, li_constrains):
    if (empty(X)):
        return False
    for i in range(len(li_f)):
        eval = li_f[i].eval(X)
        low, up = li_constrains[i]
        if ((eval.lb() < low) or (eval.ub()>up)):
            #in_it = False
            return False
    return True

In [30]:
def C_out(X,li_f, li_constrains):
    out_it = False
    for i in range(len(li_f)):
        eval = li_f[i].eval(X)
        low, up = li_constrains[i]
        if ((eval.lb()> up) or (eval.ub() < low)):
            print(i)
            out_it = True
    return out_it

In [31]:
def get_largest(X):
    max_index = 0
    for i in range(3):
        if (X[max_index].diam() < X[i].diam()):
            max_index = i
    return max_index

In [32]:
tau = 0.000001
stack = [IntervalVector([[-10,10] for _ in range(3)])]
stack_acc = []
stack_rej = []
stack_unc = []
sol = False
count = 0
while len(stack) !=0 and (not sol):
    x = stack.pop()
    count +=1
    contract(x, li_ctc)
    if not empty(x):
        if (C_in(x,li_f, li_constrains)):
            sol = True
            print("sol")
            stack_acc.append(x)
        else:
            if C_out(x, li_f, li_constrains):
                stack_rej.append(x)
            else:
                larger_index = get_largest(x)
                if x[larger_index].diam() > tau:
                    X_left, X_right  = x.bisect(larger_index)
                    if not (empty(X_left)):
                        stack.append(X_left)
                    if not (empty(X_right)):
                        stack.append(X_right)
                else:
                    stack_unc.append(x)

0
0
0
0
0
sol


In [33]:
stack_acc[0]

([2.801218788572244, 2.801219311835224] ; [-1.680632378044126, -1.680631858955939] ; [-2.123474646522542, -2.123473675635291])

In [34]:
count = 0
for node_base in node_base_connected:
    bounds = li_f[count].eval(stack_acc[0])
    print(f"Low : {bounds.lb()}    Up : {bounds.ub()}")
    print(f"Needed low : {G.get_edge_data(node,node_base)['mini']}    Needed Up : {G.get_edge_data(node,node_base)['maxi']}")
    print()
    count +=1

Low : 3.3468848592262983    Up : 3.346886033314764
Needed low : 3.280604499173894    Needed Up : 3.346886033315147

Low : 3.185788515607867    Up : 3.185789688563983
Needed low : 3.185788056352778    Needed Up : 3.250153913278571

Low : 2.469985694327776    Up : 2.469986587840734
Needed low : 2.469650991537058    Needed Up : 2.519547971363112



In [ ]:
links